

### Generación de Modelos Predictivos

Este notebook se encarga de generar modelos predictivos.  Se puede correr en Google Colab o en el iMac.

Para usarlo en Google Colab, usar tipo_a_usar='GPU'
Para usarlo en el imac, usar tipo_a_usar='CPU'



In [1]:
tipo_a_usar='CPU'

In [2]:
import pandas as pd


In [3]:
if tipo_a_usar =='GPU':
  from google.colab import drive
  drive.mount('/content/drive')

Los datos de origen es el mismo sea desde Google Colab o desde el iMac.  La diferencia es que el primero usar referencia a Google Drive y el otro acceso directo desde el sistema operativo

In [4]:
if tipo_a_usar =='GPU':
  dfbase = pd.read_pickle('/content/drive/MyDrive//Innovaciones Tecnológicas Aplicadas/Universidad Autónoma de Occidente/GoogleColab/checkPoints/DATOS_LIMPIOS.pkl')

if tipo_a_usar == 'CPU':
  dfbase = pd.read_pickle('checkpoints/DATOS_LIMPIOS.pkl')


In [5]:
df = dfbase.copy()
df = df[df['LABEL']!=-1]

X = df.drop('LABEL', axis=1)  # Reemplaza 'target_column' con el nombre de tu columna objetivo
y = df['LABEL']  # Reemplaza 'target_column' con el nombre de tu columna objetivo

In [7]:
# calculamos correlación de variables independientes
corr_matrix = X.corr()

import numpy as np
mitad_inferior = np.tril(corr_matrix, k=-1)
mitad_inferior = pd.DataFrame(mitad_inferior, columns=corr_matrix.columns, index=corr_matrix.index)

In [10]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, roc_auc_score

lista_umbral = [0.1]

for umbral in lista_umbral:
    j = 0
    variables_eliminar = set()
    for c in mitad_inferior.columns:
        i = 0
        for v in mitad_inferior[c]:
            if v > umbral:
                variables_eliminar.add(mitad_inferior.index[i])           
            i = i + 1
        j = j + 1
    print('umbral:', 
          umbral, len(mitad_inferior.columns), 
          len(variables_eliminar),
          len(mitad_inferior.columns) - len(variables_eliminar))
    
    Xnew = X.drop(list(variables_eliminar), axis=1)

    # Separar los datos en conjuntos de entrenamiento y prueba
    X_train, X_test, y_train, y_test = train_test_split(Xnew, y, test_size=0.2, random_state=42)

    # Definir los hiperparámetros a ajustar y sus posibles valores para SVM
    param_grid = {
        'C': [0.1, 1, 10],
        'kernel': ['linear', 'rbf', 'poly'],
        'degree': [2, 3, 4],
    }

    # Crear el modelo SVM
    modelo_svm = SVC(random_state=42, probability=True)

    # Realizar la búsqueda en cuadrícula con validación cruzada en el conjunto de entrenamiento
    grid_search = GridSearchCV(estimator=modelo_svm, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)
    grid_search.fit(X_train, y_train)

    print('!!!!!!! UMBRAL:', umbral)

    # Mostrar los mejores hiperparámetros y la mejor puntuación en el conjunto de entrenamiento
    print("Mejores hiperparámetros encontrados:")
    print(grid_search.best_params_)
    print("Mejor puntuación en entrenamiento:")
    print(grid_search.best_score_)

    # Obtener el modelo con los mejores hiperparámetros
    mejor_modelo_svm = grid_search.best_estimator_

    # Evaluar el modelo en el conjunto de prueba
    y_pred = mejor_modelo_svm.predict(X_test)
    accuracy_test = accuracy_score(y_test, y_pred)
    roc_auc_test = roc_auc_score(y_test, y_pred)

    # Mostrar los resultados en el conjunto de prueba
    print("Precisión (Accuracy) en prueba:", accuracy_test)
    print("ROC-AUC en prueba:", roc_auc_test)

umbral: 0.1 1953 1357 596
!!!!!!! UMBRAL: 0.1
Mejores hiperparámetros encontrados:
{'C': 10, 'degree': 2, 'kernel': 'linear'}
Mejor puntuación en entrenamiento:
0.7488658739200182
Precisión (Accuracy) en prueba: 0.7513227513227513
ROC-AUC en prueba: 0.5916562016296253


In [11]:
Xnew.shape

(6611, 596)